In [1]:
#汽车处理图像转化为模型输入
import cv2
import numpy as np
from PIL import Image
from torchvision import transforms
import torch
import torch.nn as nn
from PIL import Image
import io

In [ ]:
# Image processing for car image conversion into model input

def process_image(image):
    # Step 1: Extract the lower half of the image
    lower_half = slice_image(image)
    
    # Step 2: Convert the lower half to OpenCV format
    lower_half_cv = cv2.cvtColor(np.array(lower_half), cv2.COLOR_RGB2BGR)
    
    # Step 3: Convert to grayscale
    gray = cv2.cvtColor(lower_half_cv, cv2.COLOR_BGR2GRAY)
    
    # Step 4: Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Step 5: Apply Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)
    
    # Ensure the output image size is (64, 64)
    edges_resized = cv2.resize(edges, (64, 64))  
    
    return edges_resized

def process_image_for_model(image):
    # Convert NumPy array to PIL image
    result_image_pil = Image.fromarray(image.astype(np.uint8))

    # Ensure the image is in RGB format
    if result_image_pil.mode != 'RGB':
        result_image_pil = result_image_pil.convert('RGB')
    
    # Define the transform (already provided)
    transform = transforms.Compose([
        transforms.Resize((64, 64)),  # Ensure the image size is 64x64
        transforms.ToTensor(),        # Convert the image to Tensor format
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization
    ])
    
    # Apply transformations to the image
    image_tensor = transform(result_image_pil).unsqueeze(0)  # Add batch dimension
    return image_tensor

def process_and_prepare_for_model(image):
    # Step 1: Process the image (extract lower half and apply edge detection)
    processed_image = process_image(slice_image(image))
    
    # Step 2: Convert the processed image to the model input format
    # Since process_image returns a NumPy array, convert it to the format the model expects
    return process_image_for_model(processed_image)

# Define the model architecture (in the model set)
def convnet1(image_size=64):
    return nn.Sequential(
        # First convolutional layer, reducing the number of channels
        nn.Conv2d(3, 4, kernel_size=3, stride=1, padding=1),  # Output: 64x64x4
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 32x32x4

        # Second convolutional layer
        nn.Conv2d(4, 8, kernel_size=3, stride=1, padding=1),  # Output: 32x32x8
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 16x16x8

        # Third convolutional layer
        nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),  # Output: 16x16x16
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 8x8x16
        
        nn.Flatten(),  # Flatten the output

        # Fully connected layer
        nn.Linear(16 * 8 * 8, 128),  # Modified input size for fully connected layer: 16 * 8 * 8 = 1024
        nn.ReLU(),

        # Dropout layer to reduce overfitting
        nn.Dropout(0.3),

        # Output layer
        nn.Linear(128, 3)  # Output 3 classes
    )


In [5]:
# Rebuild the same model structure
model = convnet1(image_size=64)

# Load the saved weights
model_weights = torch.load('model_weights_3.pth')
model.load_state_dict(model_weights)
model.eval()

C:\Users\yil\AppData\Local\Temp\ipykernel_11648\3489696964.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_weights = torch.load('model_weights_3.pth')


Sequential(
  (0): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU()
  (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (9): Flatten(start_dim=1, end_dim=-1)
  (10): Linear(in_features=1024, out_features=128, bias=True)
  (11): ReLU()
  (12): Dropout(p=0.3, inplace=False)
  (13): Linear(in_features=128, out_features=3, bias=True)
)

In [ ]:
# Running car
%run car

In [ ]:
# Assign the car class to c
c = car

In [ ]:
# Print 1 through 3 using the while loop
i = 1
while i <= 3:
    i += 1
    # Open and load the image
    array = c.get()

    print(array)
    
    image = image = Image.fromarray(array)
    
    # 处理并准备图像以便输入到模型
    model_input = process_and_prepare_for_model(image)
    
    with torch.no_grad():
        output = model(model_input)

    # Process and prepare images for input into the model
    _, predicted_class = torch.max(output, 1)
    print(f"Predicted class: {predicted_class.item()}")

    if predicted_class.item() == 1:
        print(1)
    else if predicted_class.item() == 0:
        print(0)
    else:
        print(2)